### Robot Learning

### Assignment 2

Solutions are due on 25.04.2023 before the lecture.

### Introduction

Consider the following $9 \times 9$ grid world:

<img src="helpers/gridworld_sketch_legend.png" alt="Grid World" title="Grid World" width="650"/>

The agent may start in any cell that is not an obstacle nor the goal.

It can choose between eight actions, which correspond to moving to the directions 

$$a_i \in \{NW,      N,      NE,     E,     SE,    S,     SW,     W\}$$

These are indexed according to the order above, i.e. $a_0 = NW$ and $a_6 = SW$.

The agent must be careful, for the actions are non-deterministic! The agent moves with probability $0.7$ into the desired
direction, but with probability $0.1$ deviates $45^{\circ}$ to the left and with probability $0.2$ deviates $45^{\circ}$ 
to the right of the desired direction due to treacherous gusts unexpectedly sweeping the grid.

The rewards are structured as follows:

* When it reaches a blue cell, it receives a little snack of 15 points.

* When it attempts to enter a red obstacle cell, it receives -30 points and stays in the cell it came from.

* When it attempts to leave the grid, it receives -30 points and stays in the cell it came from.

* When the agent reaches the green goal cell, it receives 150 points and the episode ends.

* All other actions entering a white cell receive -1 point.

## Task 2.1

To familiarize yourself with the environment above, answer the following questions:

* The agent is at $s = (y_s, x_s) = (3, 5)$ and wants to execute $a_5$. What is the probability $P^a_{s,s'}$ for $s' =(4,6)$?


* The agent is at $s = (3, 7)$ and wants to execute $a_3$. What is the expected value of the reward?

<div style="text-align: right; font-weight:bold"> 1 + 2 = 3 Points </div>

1) Action $a_5$ is S. This action should move it to (4, 5). To get to (4, 6), it needs to execute action SE, which can be achieved by a deviation 45 degrees to the right. The probability for this action is 0.2
2) Action $a_3$ is E. This action will move it to (3, 8) with probability 0.7 and earn reward of 150 points. With probability 0.1 the agent will try to move NE to (2, 8), earn penalty of -1 point. With probability 0.2 the agent will try move SE to (4, 8), and earn penalty of -1 points.
The expected value is then 150 * 0.7 + (-1) * 0.2 + (-1) * 0.1 = 104.7

## Task 2.2)

Using the *Iterative Policy Evaluation* Algorithm, compute the value $V^{\pi}(s)$ of all accessible cells $s$ for a policy $\pi(s,a)$ that chooses with probability $0.5$ a random action and otherwise attempts to move to the right.

Intialize $V(s)$ with zero, use a discount parameter of $\gamma=0.9$ and show your results by printing your state values $V^{\pi}(s)$.

<div style="text-align: right; font-weight:bold"> 5 Points </div>

#### Note

For your convenience, you are provided the helper function *getNextStatesRewardsAndProbabilities(state, action)* which returns for a given state $s$ and an action $a$ a list of 3 -tuples of the form

$$[(s_0', R^a_{s,s_0'}, P^a_{s,s_0'}), (s_1', R^a_{s,s_1'}, P^a_{s,s_1'}), \dots]$$

where $s_i'$ are all future states with $P^a_{s,s_i'} \neq 0$. Here $s = (y, x)$ and $s_i' = (y_i', x_i')$ are both tuples of integers, $a \in {0, \dots, 7}$ is an integer, and $R^a_{s,s_i'}$, $P^a_{s,s_i'}$ are both floats.

Also, please find below some data structures which you might find helpful. Create code and text cells as necessary to present your solution!

In your implementation, $V(s)$ should be a $9 \times 9$ numpy array and $\pi(s,a)$ should be a $9 \times 9 \times 8$ numpy array, where $\sum_a \pi(s,a) = 1$ for all s!

In [2]:
import numpy as np
from helpers.utils import getNextStatesRewardsAndProbabilities
%matplotlib inline

#this is a list of all states
states = [(y,x) for y in range(9) for x in range(9)]
#this is a list of all states containing obstacles
obstacles = [(1,5), (1,7), (2,1), (2,2), (2,3), (2,4), (2,5), (2,7),\
             (3,1), (3,6), (4,3), (4,4), (4,5), (5,7), \
             (6,1), (6,2), (6,3), (6,4), (6,5), (6,6), \
             (7,8), (8,4), (8,8)]
#this is a list containing all blue cells
snacks = [(0,0), (0,1), (0,2), (0,7), (0,8), (1,8), \
         (3,2), (3,3), (3,4), (3,5), (4,2), \
         (7,1), (7,2), (7,3), (7,4), (7,5), (7,6), \
         (8,1), (8,2), (8,3), (8,5), (8,6)]
#this is a list containing all goal states
terminalStates = [(3,8)]
#this is an array containing all actions
actions = np.array([0, 1, 2, 3, 4, 5, 6, 7]) #[NW,      N,      NE,     E,     SE,    S,     SW,     W]
#example of how to unpack getNextStatesRewardsAndProbabilities(state, action):
#create dummy state and action
s_test = (0,6)
a_test = 3
#call helper function and loop over the return values
for sPrime, R, P in getNextStatesRewardsAndProbabilities(state=s_test, action=a_test):
    print('sPrime:', sPrime, 'R:', R, 'P:', P)
    
#once you have state values V, you can print them with okay'ish formatting like so:
#print("State Values:")
#print(np.around(V, 1))

sPrime: (0, 6) R: -30.0 P: 0.1
sPrime: (0, 7) R: 15.0 P: 0.7
sPrime: (0, 6) R: -30.0 P: 0.2


In [11]:
import numpy as np

# discount factor gamma
gamma = 0.9
theta = 0.0001
delta = 1

state_values = np.zeros(9 * 9)

# in my implementation, policy is a 64 * 8 array
policy = [1/16] * 8
policy[3] += 0.5

# copy policy to 64 * 8 array
policy = np.array([policy] * 81)

while delta > theta:
    delta = 0

    # for all states
    for s in range(len(states)):
        # we will get new state value
        new_s = 0

        # for all actions
        for a in range(len(actions)):

            # for all possible next states
            for sPrime, R, P in getNextStatesRewardsAndProbabilities(state=states[s], action=actions[a]):
                state_index = states.index(sPrime)
                new_s += policy[s,a]*P*(R+gamma*state_values[state_index])

        # update delta and state value
        delta = max(delta, np.abs(new_s - state_values[s]))
        state_values[s] = new_s

# reshape state values to 9 * 9 array and pretty print
state_values = state_values.reshape(9,9)

print("State Values:")
print(np.around(state_values, 1))


State Values:
[[ -97.5 -103.8 -122.3 -133.9 -142.9 -146.5 -151.5 -170.3 -210.4]
 [-108.1 -110.6 -124.  -140.3 -161.1 -139.8 -163.4 -147.5 -191.2]
 [-153.9 -119.3 -119.6 -123.3 -132.3 -125.4 -123.5  -96.7 -131.9]
 [-140.5  -93.9 -100.5 -107.2 -120.  -131.   -75.4  -48.5 -139.2]
 [-103.6  -97.5 -116.8 -117.1 -126.  -108.5  -98.7  -99.2 -120.4]
 [-105.9 -108.5 -124.9 -135.1 -139.8 -134.4 -144.6 -145.2 -172.2]
 [ -95.6  -88.5  -98.7 -108.9 -118.4 -130.9 -147.6 -170.7 -200. ]
 [ -53.2  -56.4  -68.6  -85.4 -100.9 -123.5 -151.1 -175.9 -196.6]
 [ -56.7  -60.3  -76.7 -107.3  -99.6 -126.2 -155.6 -185.  -219.7]]


## Task 2.3)

Now it is time to find a good policy. Use the *Policy Iteration* algorithm to compute the optimal value $V^*(s)$ for each accessible cell.

Retrieve the resulting optimal-policy $\pi^*(s)$. To obtain a greedy policy given $V(s)$, make use of:

$$\pi_{greedy}(s) := \operatorname{argmax}_a Q(s,a) = \operatorname{argmax}_a \sum_{s'}P_{ss'}^a\cdot[R_{ss'}^a+\gamma\cdot V(s')]$$

As implied by these terms, we recommend using intermediate state-action $Q$-values, shaped $9 \times 9 \times 8$ for this step!

Finally, present your results by printing $V^*(s)$ and using our helper function *drawPolicy()* to visualize $\pi^*(s,a)$.

<div style="text-align: right; font-weight:bold"> 5 Points </div>

In [3]:
from helpers.utils import drawPolicy
#show policy using helper function as below
#usage of the helper function, where pi is a (9,9,8) numpy array representing a deterministic policy:
#drawPolicy(pi)
#deterministic here means that one action per state has probability 1 and all others have probability 0
#this will plot arrows representing your policies into the grid world.

In [4]:
# Your code goes here

## Task 2.4)

Verify your results from the previous task by using the *Value Iteration* algorithm to compute the optimal value $V^*(s)$ for each cell. Make sure to reinitialize $V(s)$ with zero.

Finally, present your results by printing $V^*(s)$ and using our helper function *drawPolicy()* to visualize $\pi^*(s,a)$.

<div style="text-align: right; font-weight:bold"> 4 Points </div>

In [5]:
# Your code goes here

## Task 2.5)

Modify your implementation of *Value Iteration* or *Policy Iteration* to ignore the random deviations in the environment. This can be achieved by calling *getNextStatesRewardsAndProbabilities(state, action, deviation=False)*.

Present your results by printing $V^*(s)$ and using our helper function *drawPolicy()* to visualize $\pi^*(s,a)$. How and why have your state values and policy changed?

<div style="text-align: right; font-weight:bold"> 3 Points </div>

In [6]:
# Your code goes here

Please answer in this text cell.